# Metrics-Based Impact Approximation

This notebook demonstrates **metrics-based impact approximation** using `evaluate_impact()`.

## Workflow Overview

1. User provides `products.csv`
2. User configures `DATA.ENRICHMENT` section
3. User calls `evaluate_impact(config.yaml)`
4. Engine handles everything internally (adapter, enrichment, transform, model)

## Setup

In [ ]:
import json
from pathlib import Path

import yaml
from impact_engine import evaluate_impact
from online_retail_simulator import simulate

## Step 1: Create Products Catalog

In production, this would be your actual product catalog.

In [ ]:
# Create output directory
output_path = Path("output/demo_metrics_approximation")
output_path.mkdir(parents=True, exist_ok=True)

# Run simulation using config file
job_info = simulate("configs/demo_metrics_approximation_catalog.yaml")
products = job_info.load_df("products")

products_path = output_path / "products.csv"
products.to_csv(products_path, index=False)

print(f"Created: {products_path}")
print(f"Products: {len(products)}")
products

## Step 2: Configure Metrics Approximation

Configure the impact engine with:
- **ENRICHMENT**: Quality boost parameters
- **TRANSFORM**: Prepare data for approximation
- **MODEL**: `metrics_approximation` with response function

In [3]:
# Metrics approximation configuration
config = {
    "DATA": {
        "SOURCE": {
            "type": "simulator",
            "CONFIG": {
                "mode": "rule",
                "seed": 42,
                "start_date": "2024-01-01",
                "end_date": "2024-01-14",
                "path": str(products_path),
            },
        },
        "ENRICHMENT": {
            "FUNCTION": "product_detail_boost",
            "PARAMS": {
                "enrichment_fraction": 1.0,
                "enrichment_start": "2024-01-08",
                "quality_boost": 0.15,
                "seed": 42,
            },
        },
        "TRANSFORM": {"FUNCTION": "prepare_simulator_for_approximation", "PARAMS": {}},
    },
    "MEASUREMENT": {
        "MODEL": "metrics_approximation",
        "PARAMS": {"RESPONSE": {"FUNCTION": "linear", "PARAMS": {"coefficient": 0.5}}},
    },
}

config_path = output_path / "config.yaml"
with open(config_path, "w") as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"Configuration saved to: {config_path}")
print(f"\nENRICHMENT: {config['DATA']['ENRICHMENT']['FUNCTION']}")
print(f"TRANSFORM: {config['DATA']['TRANSFORM']['FUNCTION']}")
print(f"MODEL: {config['MEASUREMENT']['MODEL']}")

Configuration saved to: output/demo_metrics_approximation/config.yaml

ENRICHMENT: product_detail_boost
TRANSFORM: prepare_simulator_for_approximation
MODEL: metrics_approximation


## Step 3: Run Impact Evaluation

A single call to `evaluate_impact()` handles everything:
- Engine creates CatalogSimulatorAdapter
- Adapter simulates metrics
- Adapter generates product_details
- Adapter applies enrichment (quality boost)
- Transform extracts quality_before/quality_after
- MetricsApproximationAdapter computes impact

In [4]:
results_path = evaluate_impact(str(config_path), str(output_path / "results"))
print(f"Results saved to: {results_path}")

Results saved to: /home/peisenha/office/business/eisenhauer-io/tools/impact-engine/documentation/notebooks/output/demo_metrics_approximation/results/job-impact-engine-20260124-065000-a59bdc83/results/impact_results.json


## Step 4: Review Results

In [ ]:
with open(results_path) as f:
    results = json.load(f)

data = results["data"]
model_params = data["model_params"]
estimates = data["impact_estimates"]
summary = data["model_summary"]

print("=" * 60)
print("METRICS-BASED IMPACT APPROXIMATION RESULTS")
print("=" * 60)

print(f"\nModel Type: {results['model_type']}")
print(f"Response Function: {model_params['response_function']}")

print("\n--- Aggregate Impact Estimates ---")
print(f"Total Impact:        ${estimates['impact']:.2f}")
print(f"Number of Products:  {summary['n_products']}")

In [ ]:
from pathlib import Path

import pandas as pd

# Per-product data is now in a separate parquet file (prefixed with model type)
results_dir = Path(results_path).parent
per_product_path = results_dir / "metrics_approximation__product_level_impacts.parquet"
per_product_df = pd.read_parquet(per_product_path)

print("\n--- Per-Product Breakdown ---")
print("-" * 60)
print(f"{'Product':<20} {'Delta Quality':<15} {'Baseline':<12} {'Impact':<12}")
print("-" * 60)
for _, p in per_product_df.iterrows():
    print(
        f"{p['product_id']:<20} {p['delta_metric']:<15.4f} "
        f"${p['baseline_outcome']:<11.2f} ${p['impact']:<11.2f}"
    )

print("\n" + "=" * 60)
print("Demo Complete!")
print("=" * 60)